In [1]:
!pip install langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.0/727.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.4 MB/s eta 0:00:00


In [2]:
!pip -q install wikipedia

  Preparing metadata (setup.py) ... done


In [3]:
!pip -q install duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.0 MB/s eta 0:00:00


In [4]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 3.1 MB/s eta 0:00:00


In [5]:
import os

import openai
from langchain.agents import Tool
from langchain.utilities import WikipediaAPIWrapper
from langchain.utilities import PythonREPL
from langchain.tools import DuckDuckGoSearchRun

from langchain import OpenAI

In [21]:
os.environ["OPENAI_API_KEY"] = "####"

# Using LangChain with some Tools Wikipedia DuckDuckGo PythonRepl

### Setting up wikipedia

In [7]:
wikipedia = WikipediaAPIWrapper()

In [8]:
wikipedia.run('Langchain')

'Page: LangChain\nSummary: LangChain is a software development framework designed to simplify the creation of applications using large language models (LLMs).\n\nPage: Software development\nSummary: Software development is the process of conceiving, specifying, designing, programming, documenting, testing, and bug fixing involved in creating and maintaining applications, frameworks, or other software components. Software development involves writing and maintaining the source code, but in a broader sense, it includes all processes from the conception of the desired software through the final manifestation, typically in a planned and structured process often overlapping with software engineering. Software development also includes research, new development, prototyping, modification, reuse, re-engineering, maintenance, or any other activities that result in software products.\n\n\n\nPage: Software engineering\nSummary: Software engineering is an engineering-based approach to software de

## REPL

In [9]:
python_repl = PythonREPL()

In [10]:
python_repl.run("print(17*2)")

'34\n'

## Duck Duck Go

In [11]:
search = DuckDuckGoSearchRun()

In [12]:
search.run("Tesla stock price?")

'Get the latest Tesla Inc (TSLA) real-time quote, historical performance, charts, and other financial information to help you make more informed trading and investment decisions. Tesla, Inc. Common Stock (TSLA) Add to Watchlist Add to Portfolio Quotes Summary Real-Time After-Hours Key Data Bid Price and Ask Price The bid & ask refers to the price that an... Discover historical prices for TSLA stock on Yahoo Finance. View daily, weekly or monthly format back to when Tesla, Inc. stock was issued. The 93 analysts offering price forecasts for Tesla have a median target of 437.08, with a high estimate of 1,291.00 and a low estimate of 115.00. The median estimate represents a 35.18... TSLA: Tesla Inc - Stock Price, Quote and News - CNBC'

## Putting them together

In [22]:
llm = OpenAI(temperature=1,model_name="text-davinci-003")
#model_name="text-davinci-003", # default model
#model_name='gpt-3.5-turbo'

In [14]:
search = DuckDuckGoSearchRun()
python_repl = PythonREPL()
wikipedia = WikipediaAPIWrapper()

In [15]:
python_tool = Tool(
        name = "python repl",
        func=python_repl.run,
        description="useful for when you need to use python to answer a question. You should input python code"
    )

wikipedia_tool = Tool(
    name='wikipedia',
    func= wikipedia.run,
    description="Useful for when you need to look up a topic, country or person on wikipedia"
)

duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func= search.run,
    description="Useful for when you need to do a search on the internet to find information that another tool can't find. be specific with your input."
)


In [16]:
tools =[ duckduckgo_tool, wikipedia_tool]

## Using the agents

In [17]:
from langchain.agents import initialize_agent
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser

# conversational agent memory
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description", 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory
)

# Using the Chain

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = LLMChain(llm=chat, prompt=chat_prompt)

# Using Prompt template

In [24]:
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="List five fitness sport in noun format which is good for {subject} f.\n{format_instructions}",
    #template="List five exercise similar with {subject} f.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)



_input = prompt.format(subject="flexibility") # {키워드}한 운동 추천
output_text=zero_shot_agent(_input)
output_kor=chain.run(input_language="English", output_language="Korean", text=output_text["output"])
answer = output_parser.parse(output_kor)[:5]
answer



> Entering new AgentExecutor chain...
 I should research some of the exercise activities that promote flexibility
Action: DuckDuckGo Search
Action Input: "Fitness activities for flexibility"
Observation: Try It. This 15-Minute Yoga Flow Eases Tight Muscles and Builds Total-Body Flexibility. 2. Pilates for Flexibility. Pilates is one of the best exercises for gaining flexibility. It lets you train several muscle groups at the same time and, because it's a low-impact exercise, it's gentle on your joints, per the Mayo Clinic. Try adding the stretches for flexibility that Atkins demos below—everything from lower-back stretches and upper-back stretches to hamstring and quad stretches, to moves that loosen up your... Flexibility exercise is one of the four types of exercise along with strength, balance and endurance.Ideally, all four types of exercise would be included in a healthy workout routine and AHA provides easy-to-follow guidelines for endurance and strength-training in its Recomme

['요가', '필라테스', '유연성 운동', '정적 스트레칭', '동적 스트레칭.']

* cardiovascular health : 심폐지구력
* muscular endurance : 근지구력
* flexibility : 유연성